In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from split_llm.glm6b.wrapped_layer import Attention_GLM_Wrapped, copy_attention
from split_llm.glm6b.utils import generate_position_ids

/home/zf/anaconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from llm_bases.chatglm6b import ChatGML6B
glm = ChatGML6B()

Loading checkpoint shards: 100%|██████████| 8/8 [00:11<00:00,  1.46s/it]


In [3]:
transformer_layer = glm.condgen.transformer.layers[0].float()
attn_wrapped = Attention_GLM_Wrapped(4096, 32, 0)
copy_attention(transformer_layer, attn_wrapped)
attn_wrapped.cuda()

Attention_GLM_Wrapped(
  (positional_embedding): GLMPositionalEmbedding()
)

In [4]:
import torch
x = torch.normal(0, 1, [10, 1, 4096]).cuda()

In [55]:
# Generate the hidden representations in a normal transformer layer

h_qkv = x @ attn_wrapped.qkv_weight.T + attn_wrapped.qkv_bias
qs, ks, vs = h_qkv.view(-1, 1, 32, 128 * 3).chunk(3, dim=-1)
qs, ks = attn_wrapped.positional_embedding(qs, ks, generate_position_ids(10, 10).cuda())
scores = attn_wrapped.generate_logit_scores(qs, ks)
softmax_scores = attn_wrapped.generate_softmax_scores(scores, dim=1)
weighted_v = attn_wrapped.generate_weighted_values(softmax_scores, vs)

In [82]:
try:
    import sys
    del sys.modules['split_llm.glm6b.utils']
    del sys.modules['split_llm.protocols.base_protocols']
    del sys.modules['split_llm.protocols.ss_mul_with_memory']
    del sys.modules['split_llm.protocols.element_wise']
    del sys.modules['split_llm.glm6b.secure_inference']
    del sys.modules['split_llm.glm6b.wrapped_layer']
    del sys.modules['split_llm.common.torch_utils']
    print("delete complete!")
except Exception as e:
    print(e)
from split_llm.glm6b.secure_inference import GLMAttentionProtocol
from split_llm.common.torch_utils import relative_error

delete complete!


In [83]:
transformer_layer = glm.condgen.transformer.layers[0].float()
attn_wrapped = Attention_GLM_Wrapped(4096, 32, 0)

copy_attention(transformer_layer, attn_wrapped)

In [84]:
from split_llm.common.communication import Communication, Node, SimulatedCommunication
communication = SimulatedCommunication(["n0", "n1", "n2"])
communication.new_stage("Test")

n0 = Node(communication, "n0")
n1 = Node(communication, "n1")
n2 = Node(communication, "n2")

In [85]:
n0.space.attentions = [attn_wrapped.cuda()]

In [86]:
protocol = GLMAttentionProtocol(
    n0, n1, n2, 0, 10, device="cuda"
)
protocol.prepare()
protocol.offline_execute(10)

In [87]:
x0 = torch.normal(0, 1, [10, 1, 4096]).cuda()
x1 = x - x0
n0.storage[f"{protocol.name}:x0"] = x0
n1.storage[f"{protocol.name}:x1"] = x1


In [88]:
# Test computing QKV
protocol.online_step_qkv()

In [89]:
qkv_computed = n0.storage[f"{protocol.name}:h0"] + n1.storage[f"{protocol.name}:h1"]
print(f"QKV error: {relative_error(qkv_computed, h_qkv):.5f}")

QKV error: 0.00046


In [90]:
# Test computing dot-product
protocol.online_step_dot_product()


In [91]:
computed_scores = n0.storage[f"{protocol.name}:s0"] + n1.storage[f"{protocol.name}:s1"]
print(f"Scores error: {relative_error(computed_scores, scores):.5f}")

Scores error: 0.00048


In [92]:
# Test computing softmax scores
protocol.online_step_softmax()

In [93]:
computed_softmax_scores = n0.storage[f"{protocol.name}:s0"] + n1.storage[f"{protocol.name}:s1"]
print(f"Softmax Scores error: {relative_error(computed_softmax_scores, softmax_scores):.5f}")

Softmax Scores error: 0.00056


In [94]:
# Test compute weighted values
protocol.online_step_weighted_v()

In [96]:
computed_v = n0.storage[f"{protocol.name}:h0"] + n1.storage[f"{protocol.name}:h1"]
print(f"Weighted V error: {relative_error(computed_v, weighted_v):.5f}")

Weighted V error: 0.00055


tensor([[[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
          1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
          1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
          1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000]],

        [[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
          1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
          1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
          1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000]],

        [[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
          1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
          1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
          1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000]],

        [[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
          1.0000, 1.0000, 1.0

In [21]:
softmax_scores

tensor([[[[0.0220, 0.0767, 0.0130,  ..., 0.1028, 0.0860, 0.0713]],

         [[0.0172, 0.0446, 0.1339,  ..., 0.1029, 0.1990, 0.1635]],

         [[0.0431, 0.0183, 0.0397,  ..., 0.0485, 0.0528, 0.0405]],

         ...,

         [[0.3899, 0.0645, 0.1712,  ..., 0.1365, 0.1864, 0.0897]],

         [[0.1867, 0.2289, 0.2115,  ..., 0.1220, 0.0426, 0.0963]],

         [[0.0352, 0.0694, 0.0859,  ..., 0.1079, 0.0551, 0.0420]]],


        [[[0.0851, 0.0858, 0.0490,  ..., 0.1082, 0.1843, 0.0451]],

         [[0.0763, 0.0539, 0.0035,  ..., 0.1005, 0.0780, 0.0459]],

         [[0.0727, 0.1401, 0.2799,  ..., 0.1725, 0.0887, 0.0848]],

         ...,

         [[0.0234, 0.1186, 0.1014,  ..., 0.0566, 0.1009, 0.1451]],

         [[0.0954, 0.0188, 0.0608,  ..., 0.1078, 0.0903, 0.1563]],

         [[0.0725, 0.0369, 0.0479,  ..., 0.0787, 0.2290, 0.1117]]],


        [[[0.0211, 0.0567, 0.1949,  ..., 0.1389, 0.1557, 0.0738]],

         [[0.1250, 0.0632, 0.0367,  ..., 0.0437, 0.0736, 0.0898]],

         [[0.0